# Evaluation of experiments - union - DREAMS
fistly define the UUID of the experiment to use

In [1]:
experimentId="68877408-cc71-4aee-8777-09b36e149a50"

-------------------------------------------------------

In [2]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import sklearn.metrics as skm

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"\\COGNITION"
dreamspath=datapath+"\\DREAMS"
masspath=datapath+"\\MASS"


## load dreams

In [3]:
samplerate=200  #Should rethink this

In [4]:
signals, annotations, signalsMetadata = loadDREAMSSpindles(dreamspath)

SubjectId: 0001--------------
resampling from 100 to 200
SubjectId: 0003--------------
resampling from 50 to 200
SubjectId: 0006--------------
duration discrepancy, removing last 0.7 seconds


In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

764

In [6]:
minDuration=0.3
maxDuration=2.9
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

764

In [7]:
signalsMetadata.head()

,filename,channel,subjectId,duration,samplerate,database
0,excerpt1.txt,C3-A1,0001,1800,200,DREAMS
1,excerpt2.txt,CZ-A1,0002,1800,200,DREAMS
2,excerpt3.txt,C3-A1,0003,1800,200,DREAMS
3,excerpt4.txt,CZ-A1,0004,1800,200,DREAMS
4,excerpt5.txt,CZ-A1,0005,1800,200,DREAMS


## Load experiment results

In [8]:
experimentModels, featureSelection = loadExperiment(experimentId,datapath)

In [9]:
experimentModels.head()

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0006]",[0002],0001,83b7a320-06e0-4b5f-a99f-7863b21ac84b,0.018348
1,0001,E1,[0001],"[0001, 0003, 0005]",[0006],0002,de0568df-657a-4cef-b14e-7d0cae36ca8e,0.016172
2,0001,E1,[0001],"[0001, 0002, 0006]",[0005],0003,3bcdc5c8-b4d4-4f58-a892-fcc66da68ef3,0.026374
3,0001,E1,[0001],"[0001, 0002, 0006]",[0003],0005,3c4bdd12-6869-41a4-a9b0-70c68730415d,0.026374
4,0001,E1,[0001],"[0001, 0003, 0005]",[0002],0006,a2fe24ce-b6bb-43f9-a58e-b25f9e3630bc,0.016172


In [10]:
featureSelection

,window,characteristic,bandName,score
0,2,hjortActivity,beta2,69.0
1,1,hjortActivity,sigma,63.0
2,2,hjortActivity,beta1,42.0
3,2,hjortActivity,delta2,41.0
4,1.5,sigmaIndex,broadband,39.0
5,2,sigmaIndex,broadband,39.0
6,2,hjortActivity,theta,38.0
7,2,hjortMobility,beta1,37.0
8,2,hjortActivity,alpha,35.0
9,1,sigmaIndex,broadband,34.0


## Optimal hyperparameter estimation with UNION criterium
we test the optimal points for the prediction threshold and number of boost iterations in the different validation groups

In [11]:
experimentModels=experimentModels[experimentModels.criteriumName=='union'].reset_index(drop=True)
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0003,union,"[0001, 0002]","[0003, 0005, 0006]",[0002],0001,a7cb461b-4f33-4085-89b3-1f619ddca728,0.047350
1,0003,union,"[0001, 0002]","[0001, 0003, 0005]",[0006],0002,874f5503-0e4f-4cc8-b19b-6517f13627f2,0.051389
2,0003,union,"[0001, 0002]","[0001, 0002, 0006]",[0005],0003,36d900b6-57fb-4984-9c7e-16bc86690129,0.058704
3,0003,union,"[0001, 0002]","[0001, 0002, 0006]",[0003],0005,bd6db5d7-e220-4ec8-8aaa-386ccbd7d59c,0.058704
4,0003,union,"[0001, 0002]","[0001, 0003, 0005]",[0002],0006,a9030ba3-3169-490e-b375-21663205d61d,0.051389


In [29]:
auxThres=pd.DataFrame({
    'hyperThres':[0.3,0.4,0.5,0.6,0.7,0.8]
})
auxDepth=pd.DataFrame({
    'hyperDepth':[5,10,15,20,30,40]
})
auxDuration=pd.DataFrame({
    'hyperDuration':[0.3]
})
auxClose=pd.DataFrame({
    'hyperClose':[0.25]
})
auxConfidence=pd.DataFrame({
    'hyperConfidence':[0]
})

hyperParams=pd.merge(auxThres,auxDepth,how='cross')
hyperParams=pd.merge(hyperParams,auxDuration,how='cross')
hyperParams=pd.merge(hyperParams,auxClose,how='cross')
hyperParams=pd.merge(hyperParams,auxConfidence,how='cross')

hyperParams

,hyperThres,hyperDepth,hyperDuration,hyperClose,hyperConfidence
0,0.3,5,0.3,0.25,0
1,0.3,10,0.3,0.25,0
2,0.3,15,0.3,0.25,0
3,0.3,20,0.3,0.25,0
4,0.3,30,0.3,0.25,0
5,0.3,40,0.3,0.25,0
6,0.4,5,0.3,0.25,0
7,0.4,10,0.3,0.25,0
8,0.4,15,0.3,0.25,0
9,0.4,20,0.3,0.25,0


In [30]:
def getCoordAndLenList(labelVector):
    objects=ndi.find_objects(ndi.label(labelVector)[0])
    coords=list(map(lambda x: (x[0].start,x[0].stop,x[0].stop-x[0].start),objects))
    return coords
    
def rawAndProcessedToConfidenceAnnotations(raw,processed,samplerate):
    #calculate the number of true raw outputs in each detection
    labels=ndi.label(processed)
    actualPredicted=np.zeros((labels[1],))
    for i in np.arange(0,labels[1])+1:
        actualPredicted[i-1]=np.sum(raw*(labels[0]==i))
    #calculate the confidences
    outCoords=getCoordAndLenList(processed)
    confidences=[]
    for coords,actual in zip(outCoords,actualPredicted):
        confidences.append(actual/coords[2])
    #create a data frame
    confidenceDf=pd.DataFrame({
        'startInd':[x[0] for x in outCoords],
        'stopInd':[x[1] for x in outCoords],
        'duration':[x[2]/samplerate for x in outCoords],
        'confidence':confidences
    })
    return confidenceDf

In [31]:
def getIou(coordA,coordB):
    if (coordA[1]<coordB[0])|(coordB[1]<coordA[0]): #NOT INTERSECTION
        iou=0
    else:   #INTERSECTION
        inter=np.min((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        union=np.max((coordA[1]-coordB[0],coordB[1]-coordA[0]))
        iou=inter/union
    return iou

def annotationPairToMetrics(annotations,detections,thresIoU=0.2):
    #get the coords
    gtCoords=zip(annotations.startInd,annotations.stopInd)
    outCoords=zip(detections.startInd,detections.stopInd)
    #calculate the iou vector
    iouVector=np.array(list(itt.starmap(getIou,itt.product(gtCoords,outCoords))))
    #reshape to a matrix
    iouMatrix=iouVector.reshape(len(annotations),len(detections))
    #binarize
    binarized=iouMatrix>thresIoU
    #calculateF1
    outF1=(np.sum(np.max(binarized,axis=0))+np.sum(np.max(binarized,axis=1)))/(len(annotations)+len(detections))
    recall=np.sum(np.max(binarized,axis=1))/len(annotations)
    precision=np.sum(np.max(binarized,axis=0))/len(detections)
    return outF1,recall,precision

In [32]:
hyperExperimentModels=[]
hyperMeanF1=[]
hyperStdF1=[]

for ind_hyper,row_hyper in hyperParams.iterrows():
    print(ind_hyper)
    thisExperimentModels=experimentModels.copy()
    hyperThres=row_hyper.hyperThres
    hyperDepth=int(row_hyper.hyperDepth)
    hyperDuration=row_hyper.hyperDuration
    hyperClose=row_hyper.hyperClose
    """ hyperConfidence=row_hyper.hyperConfidence """
    
    meanF1=[]
    meanPrecision=[]
    meanRecall=[]

    stdF1=[]
    stdPrecision=[]
    stdRecall=[]

    for ind,row in thisExperimentModels.iterrows():
        #load model
        model=loadBooster(row.modelId,experimentId,datapath)
        #initialise lists
        rawF1s=[]
        rawPrecisions=[]
        rawRecalls=[]

        f1s=[]
        precisions=[]
        recalls=[]

        #iterate validation subjects
        for valSubjectId in row.val:
            #Define annotations criterium
            usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
            #Load features and labels
            valFeatures=loadFeatureMatrix([valSubjectId],featureSelection,signalsMetadata,samplerate,datapath)
            valLabels=loadLabelsVector([valSubjectId],usedAnnotations,signalsMetadata,samplerate)
            #Predict
            valDMatrix=xgb.DMatrix(data=valFeatures)
            probabilities=model.predict(valDMatrix,iteration_range=(0,hyperDepth))
            raw=probabilities>=hyperThres
            #Processed labels
            processed=labelingProcess(raw,hyperClose,hyperDuration,samplerate)
            """ #Detections with confidence
            detections=rawAndProcessedToConfidenceAnnotations(raw,processed,samplerate)
            #Confidence threshold
            detections=detections[detections.confidence>hyperConfidence].reset_index(drop=True) """
            gtAnnotations=labelVectorToAnnotations(valLabels,samplerate)
            detections=labelVectorToAnnotations(processed,samplerate)
            #Metrics
            f,r,p=annotationPairToMetrics(gtAnnotations,detections)
            
            #Metric appends
            f1s.append(f)
            precisions.append(p)
            recalls.append(r)

        #statistics of the metrics over the subjects of the validation set
        meanF1.append(np.mean(f1s))
        meanPrecision.append(np.mean(precisions))
        meanRecall.append(np.mean(recalls))

        stdF1.append(np.std(f1s))
        stdPrecision.append(np.std(precisions))
        stdRecall.append(np.std(recalls))
        
    thisExperimentModels['meanF1']=meanF1
    thisExperimentModels['meanPrecision']=meanPrecision
    thisExperimentModels['meanRecall']=meanRecall
    
    thisExperimentModels['stdF1']=stdF1
    thisExperimentModels['stdPrecision']=stdPrecision
    thisExperimentModels['stdRecall']=stdRecall

    hyperExperimentModels.append(thisExperimentModels)
    hyperMeanF1.append(np.mean(thisExperimentModels['meanF1']))
    hyperStdF1.append(np.std(thisExperimentModels['meanF1']))
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [33]:
hyperParams['meanMeanF1']=hyperMeanF1
hyperParams['stdMeanF1']=hyperStdF1

In [34]:
optimalInd=hyperParams[hyperParams.meanMeanF1==np.max(hyperParams.meanMeanF1)].index[0]
print("maximal mean score at:")
optimal=hyperParams.iloc[optimalInd]
optimal

maximal mean score at:


hyperThres          0.500000
hyperDepth         15.000000
hyperDuration       0.300000
hyperClose          0.250000
hyperConfidence     0.000000
meanMeanF1          0.722545
stdMeanF1           0.050125
Name: 14, dtype: float64

## Graphical representation of the optimal point

In [35]:
aux=hyperParams[(hyperParams.hyperDepth==optimal.hyperDepth)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperThres',y='meanMeanF1',error_y='stdMeanF1')

In [36]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDuration==optimal.hyperDuration)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperDepth',y='meanMeanF1',error_y='stdMeanF1')

In [22]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperClose==optimal.hyperClose)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperDuration',y='meanMeanF1',error_y='stdMeanF1')

In [23]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperDuration==optimal.hyperDuration)&(hyperParams.hyperConfidence==optimal.hyperConfidence)].reset_index(drop=True)
px.scatter(aux,x='hyperClose',y='meanMeanF1',error_y='stdMeanF1')

In [23]:
aux=hyperParams[(hyperParams.hyperThres==optimal.hyperThres)&(hyperParams.hyperDepth==optimal.hyperDepth)&
                (hyperParams.hyperDuration==optimal.hyperDuration)&(hyperParams.hyperClose==optimal.hyperClose)].reset_index(drop=True)
px.scatter(aux,x='hyperConfidence',y='meanMeanF1',error_y='stdMeanF1')